In [ ]:
import os
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import joblib as jb
from pca import PCA


# Carregar o arquivo .mat
mat = scipy.io.loadmat('taira_R100_yf.mat')


data_u = mat['u']

data_u_x = data_u['x'][0, 0]
data_u_y = data_u['y'][0, 0]

# Acessar a estrutura 'dat'
dat = mat['dat']

# Acessar a estrutura 'bat' dentro de 'dat'
bat = dat['bat'][0, 0]

# Extrair as colunas w, f, t, u, e v de 'bat'
w = bat['w']
f = bat['f']
t = bat['t']
u = bat['u']
v = bat['v']

# Converter t para um array 1D de valores float
t_values = np.array([time[0][0] for time in t[0]])
# Verificar os tamanhos dos vetores u e v
print("u shape:", u.shape)  # Esperado: (1, 400)
print("v shape:", v.shape)  # Esperado: (1, 400)
print("t shape:", t.shape)  # Esperado: (1, 400)
print("data_u_x shape:", data_u_x.shape)  # Esperado: (300, 200)
print("data_u_y shape:", data_u_y.shape)  # Esperado: (300, 200)
print(u[0, 1].shape) # Esperado: (300, 199)
print(v[0, 1].shape) # Esperado: (299, 200)
# Inicializar listas para armazenar resultados
u_medio_list = []
v_medio_list = []
V_medio_list = []

# Iterar sobre cada elemento (matriz) de 'u' e 'v'
for i in range(t.shape[1]):
    u_i = u[0, i]  # Acessar a matriz u correspondente ao tempo i
    v_i = v[0, i]  # Acessar a matriz v correspondente ao tempo i

    # Calcular a média entre as linhas adjacentes de u_i
    u_medio = (u_i[:-1, :] + u_i[1:, :]) / 2
    u_medio_list.append(u_medio)
    
    # Calcular a média entre as colunas adjacentes de v_i
    v_medio = (v_i[:, :-1] + v_i[:, 1:]) / 2
    v_medio_list.append(v_medio)
    
    # Calcular V_medio para o tempo i
    V_medio = np.sqrt(u_medio**2 + v_medio**2)
    V_medio_list.append(V_medio)

# Converter as listas de resultados em arrays NumPy
u_medio_array = np.array(u_medio_list)
v_medio_array = np.array(v_medio_list)
V_medio_array = np.array(V_medio_list)

# Verificar as novas formas
print("Shape de u_medio_array:", u_medio_array.shape)
print("Shape de v_medio_array:", v_medio_array.shape)
print("Shape de V_medio_array:", V_medio_array.shape)
# Inicializar a lista para armazenar os vetores q_t
Q_list = []

# Iterar sobre cada tempo t para construir os vetores q_t
for i in range(t.shape[1]):
    u_medio_flat = u_medio_array[i].flatten()
    v_medio_flat = v_medio_array[i].flatten()
    
    q_t = np.concatenate((u_medio_flat, v_medio_flat))
    Q_list.append(q_t)

# Converter a lista de vetores q_t em uma matriz NumPy
Q = np.array(Q_list).T

# Verificar a forma da matriz Q
print("Shape da matriz Q:", Q.shape)

X = Q.T
# Suponha que Q seja sua matriz de dados com forma (119002, 400)
# Transpor a matriz para aplicar PCA ao longo do tempo
Q_transposed = Q.T

# Inicializar e aplicar PCA
n_components_ = 10
pca = PCA(n_components=n_components_)
Q_reduced = pca.fit_transform(Q_transposed)

# Forma da matriz reduzida
print("Shape of Q_reduced:", Q_reduced.shape)  # Deve imprimir (400, 400)

# Variância explicada
exp_var_pca = np.absolute(pca.explained_variance_ratio_)[:8]
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
# Calcular a porcentagem da variância não explicada
remaining_variance = (1 - cum_sum_eigenvalues) * 100

# Gráfico de variância explicada
plt.figure(figsize=(12, 8))
plt.bar(range(len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Razão individual de variância')
plt.step(range(len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid', label='Soma cumulativa das razões de variância', color='red')
plt.ylabel('Razão de variância')
plt.xlabel('Índice da componente principal')
plt.legend(loc='best')
plt.grid()

# Adicionar anotações para a porcentagem da variância não explicada
for i in range(len(remaining_variance)):
    plt.annotate(f'{remaining_variance[i]:.2f}%', (i, cum_sum_eigenvalues[i]), textcoords="offset points", xytext=(0, 10), ha='center', color='blue')

plt.tight_layout()
plt.savefig("pca_components.png", dpi=400, format="png", bbox_inches='tight')
plt.show()


# Plotar cada componente principal ao longo do tempo em figuras separadas
for i in range(n_components_):
    plt.figure(figsize=(8, 6))
    plt.plot(t_values, Q_reduced[:, i], label=f'PC {i+1}')
    plt.xlabel('Tempo')
    plt.ylabel('Valor da Componente Principal')
    plt.title(f'Componente Principal {i+1} ao longo do Tempo')
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"pca_time_series_PC{i+1}.png", dpi=200, format="png", bbox_inches='tight')
    plt.show()

X_transposed = Q_reduced.T

V = defaultdict(list)

for row in range(X_transposed.shape[0]):
    for col in range(X_transposed.shape[1]):
        V[f'column_{col}_real'].append(X_transposed[row, col].real)
        V[f'column_{col}_imag'].append(X_transposed[row, col].imag)

V = pd.DataFrame(V)
V.to_csv("pca_reduced_data.csv", index=False)
jb.dump(pca, "pca_complex.pkl.z")